In [7]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [8]:
load_dotenv(override=True)

openai_model = "gpt-4o-mini"
llm = ChatOpenAI(model=openai_model, temperature=0.1)

In [ ]:
article_text = """AI is a useful tool for creative purposes. It can write text, create images and videos and act as a useful
assistant to the creator. Rather than thinking of AI as a threat, we need to embrace its power to be more productive
and experiment more and delegate the mundaneness to our digital assistant."""

In [10]:
system_message = SystemMessagePromptTemplate.from_template("""
You are a helpful assistant that helps generate article titles.
""")

In [14]:
user_prompt = HumanMessagePromptTemplate.from_template(
    """You are looking at an article text:
    ---
    {article_text}
    --- 
    Generate a suitable title for the same.
    The tone of the title should be comic yet catchy.""",
    input_variables=["article_text"]
)

In [15]:
# Using input_variables in the prompt template
user_prompt.format(article_text=article_text)

HumanMessage(content='You are looking at an article text:\n    ---\n    AI is a useful tool for creative purposes. It can write text, create images and videos and act as a useful\nassistant to the creator. Rather than thinking of AI as a threat, we need to embrace its power to be more productive\nand experiment more and delegate the mundaneness to our digital assistant.\n    --- \n    Generate a suitable title for the same.\n    The tone of the title should be comic yet catchy.', additional_kwargs={}, response_metadata={})

In [16]:
first_prompt = ChatPromptTemplate.from_messages([system_message, user_prompt])

In [17]:
print(first_prompt.format(article_text=article_text))

System: 
You are a helpful assistant that helps generate article titles.

Human: You are looking at an article text:
    ---
    AI is a useful tool for creative purposes. It can write text, create images and videos and act as a useful
assistant to the creator. Rather than thinking of AI as a threat, we need to embrace its power to be more productive
and experiment more and delegate the mundaneness to our digital assistant.
    --- 
    Generate a suitable title for the same.
    The tone of the title should be comic yet catchy.


Now my prompt is ready, I have defined my model as well as input. Now its time to feed the prompt to the model.

# Create chain using prompt and LLM.

The chain will do the following:

1. Format the prompt
2. generate using LLM
3. Output


Chain is created using LCEL.

In the following chain the output of LLM is an AIMessage

In [20]:
chain_one = ({"article_text": lambda x: x["article_text"]} 
             | first_prompt
             | llm
             | {"article_title": lambda x: x.content})

In [38]:
title_output = chain_one.invoke({"article_text": article_text})["article_title"]

In [39]:
title_output

'"AI: Your New Creative Sidekick – Because Who Needs Mundane When You Have a Digital Genie?"'

* So we have generated article title using LLM.

* Next we'll patch this title with the article and generate an article summary.

In [46]:
second_system_prompt = SystemMessagePromptTemplate.from_template("""You are a helpful assistant who is able to help in building good articles.
                                                                 """)


second_user_prompt = HumanMessagePromptTemplate.from_template("""
You have access to an article: 
{article_text} alongwith its title: {article_title}. 
Generate a crisp summary for the article using this information. Make sure that the summary is less than 100 characters.
The description should be in a SEO compatible format and suitable to show up on Google when this appears in search results""",
input_variables = ["article_text", "article_title"])


second_prompt = ChatPromptTemplate([second_system_prompt, second_user_prompt])

In [47]:
chain_two = (
    {
        "article_text": lambda x: x["article_text"],
        "article_title": lambda x: x["title"]
    }
    | second_prompt
    | llm
    | {"article_description": lambda x: x.content}
)

In [48]:
chain_two.invoke({"article_text": article_text, "title": title_output })

{'article_description': 'Discover how AI enhances creativity by handling mundane tasks, making you more productive.'}